In [1]:
import json
import re

with open('credentials.json') as file:
    credentials = json.load(file)

In [2]:
import requests
from datetime import datetime, timedelta

def get_sessions(session_type, req_from = 0):
    '''
    Returns all sessions based on sessiontype.
    '''

    url = "https://events.rainfocus.com/api/search"

    payload =   {
                    'search.sessiontype': session_type,
                    'type': 'session',
                    'catalogDisplay': 'list',
                    'browserTimezone': 'Europe%2FParis',
                    'from': req_from
                }

    headers = {
        'authority': 'events.rainfocus.com',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9,fr-FR;q=0.8,fr;q=0.7',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'origin': 'https://www.ciscolive.com',
        'referer': 'https://www.ciscolive.com/',
        'rfapiprofileid': credentials['rfapiprofileid'],
        'rfauthtoken': credentials['rfauthtoken'],
        'rfwidgetid': credentials['rfwidgetid'],
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.json()

class Session():

    def __init__(self, session_json):
        self.type = session_json['type']
        try:
            self.start = datetime.strptime(session_json['times'][0]['utcStartTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
            self.end = datetime.strptime(session_json['times'][0]['utcEndTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
        except KeyError:
            self.start = 'Null'
            self.end = 'Null'
        self.day_name = session_json['times'][0]['dayName']
        self.name = re.sub(r"", "", session_json['title'])
        self.id = session_json['code']
        match int(self.id.split('-')[1][0]):
            case 1: self.level = 'Introductory'
            case 2: self.level = 'Intermediate'
            case 3: self.level = 'Advanced'
            case 4: self.level = 'General'
        self.capacity = int(session_json['times'][0]['capacity'])
        self.seats_remaining = int(session_json['times'][0]['seatsRemaining'])
        self.waitlist_remaining = int(session_json['times'][0]['waitlistRemaining'])
        self.room_id = session_json['times'][0]['roomId']
        self.room = session_json['times'][0]['room']
        
    def __str__(self):
        return self.name

In [99]:
if __name__ == '__main__':
    
    with open('credentials.json') as file:
        credentials = json.load(file)

    sessions = []
    session_types = ['Technical_seminar', 'BRK', 'Product-or-Strategy-Overview', 'Innovation_talk', 'Partner Session', 'DevNet', 'Walk-in Lab', 'Instructor Led Lab', 'Additional Programs']

    for session_type in session_types:

        full_sessions = 0
        
        sessions_json = get_sessions(session_type)

        req_from = int(sessions_json['sectionList'][0]['from'])
        num_items = int(sessions_json['sectionList'][0]['numItems'])
        total = int(sessions_json['sectionList'][0]['total'])
        size = int(sessions_json['sectionList'][0]['size'])

        while req_from + num_items < total:
            req_from += size
            sessions_json_temp = get_sessions(session_type, req_from=req_from)

            for session in sessions_json_temp['items']:
                sessions_json['sectionList'][0]['items'].append(session)


        for session in sessions_json['sectionList'][0]['items']:
            sessions.append(Session(session))

In [100]:
sessions_dict = {}

for session in sessions:
    sessions_dict[session.id] = session.name

In [101]:
import pandas as pd

df = pd.read_excel('french_customers_sessions.xlsx')

In [102]:
class Customer():

    def __init__(self, df_line) -> None:
        self.name = df_line.iloc[1] + ' ' + df_line.iloc[2]
        self.company = df_line.iloc[3]
        self.sessions = df_line.iloc[7].split(' | ')

In [103]:
customers = []

for index, row in df.iterrows():
    customers.append(Customer(row))

In [104]:
sessions_attended = []

for customer in customers:
    sessions_attended.extend(customer.sessions)

sessions_attended = list(set(sessions_attended))
print(sessions_attended)

['BRKOPT-2837', 'BRKDCN-2938', 'BYODCRT-2464', 'BRKNWT-2210', 'BRKMER-2399', 'TECIPV-2265', 'BRKAPP-1154', 'TECSPG-2014', 'BRKMER-1752', 'BRKPAR-3569', 'BRKCCT-2724', 'BRKARC-2882', 'BRKSPG-2133', 'BRKTRS-2034', 'BRKSPG-2355', 'BRKOPT-2705', 'PSOCX-2113', 'DEVNET-2572', 'BRKCRT-2008', 'TECSPG-3204', 'DEVNET-2527', 'TSCSPG-2010', 'LTRCRT-2157', 'BRKSEC-2044', 'VR-1001', 'ADD-3127', 'DEVNET-1487', 'DEVNET-2628', 'BRKENT-3412', 'BRKOPT-1005', 'BRKOPS-2068', 'PSOIOT-1004', 'BRKSEC-3320', 'BRKOPS-2312', 'CSSSPG-2572', 'LABOPS-1507', 'DEVNET-1700', 'BRKSPG-2835', 'BRKOPT-2578', 'LTRMPL-2208', 'DEVNET-2094', 'CISCOU-2672', 'DEVWKS-1176', 'LTRMPL-2116', 'BRKSPG-2028', 'PSOSEC-1213', 'BRKDCN-3982', 'LTROPS-2835', 'DEVNET-2273', 'BRKSPG-3050', 'BRKSEC-3129', 'BRKSPG-2029', 'BRKPAR-3304', 'DEVWKS-2919', 'PSOMS-1330', 'BRKOPS-2431', 'BRKSEC-2051', 'BRKOPS-2176', 'TECSPG-2435', 'BRKOPS-2376', 'BRKDCN-1601', 'BRKSPG-3624', 'BRKSEC-2644', 'BRKPAR-2906', 'BRKSPG-2031', 'TECIPV-2000', 'LTRSEC-2272', 'B

In [112]:
sessions_customers_dict = {}

for session in sessions_attended:
    sessions_customers_dict[session + ' - ' + sessions_dict.get(session, 'unknown')] = []

In [113]:
for customer in customers:
    for session in customer.sessions:
        sessions_customers_dict[session + ' - ' + sessions_dict.get(session, 'unknown')].append(customer.name + f' ({customer.company})')

In [114]:
sessions_customers_dict

{'BRKOPT-2837 - Scaling Enterprise Networks with Routed Optical Networking': ['Thomas Delaby (France-IX)'],
 'BRKDCN-2938 - Intersight Managed Mode (IMM) for UCS Manager admins': ['Bastien Fine (Adista)'],
 'BYODCRT-2464 - unknown': ['zakariaa FODIL (LINKT)'],
 'BRKNWT-2210 - Predicting Networks are THERE !!': ['Mathieu Bonnin (Axione)',
  'Frederic Faraux (Claranet)'],
 'BRKMER-2399 - Meraki Wireless from a Troubleshooter Perspective': ['Stanislas de Goriainoff (SEWAN)'],
 'TECIPV-2265 - IPv6 in your Network': ['Bastien Fine (Adista)',
  'Thomas Delaby (France-IX)',
  'Vittorio Mihailescu (France-IX)'],
 'BRKAPP-1154 - Do Tell About OTel: An Introduction to OpenTelemetry and How AppDynamics is Embracing It': ['Pierre Etancelin (ALTITUDE INFRA)',
  'Mathieu Bonnin (Axione)'],
 'TECSPG-2014 - Cisco Converged SDN Transport': ['Jean-Michel Dilly (Adista)'],
 'BRKMER-1752 - Experience the Journey to IPv6-Only With Cisco Meraki': ['Gilles Friang (SEWAN)',
  'Stanislas de Goriainoff (SEWAN)'

In [125]:
df_final = pd.DataFrame({'Sessions':[], 'Customers':[]})

df_final

,Sessions,Customers


In [133]:
for key, value in sessions_customers_dict.items():
    new_row = pd.DataFrame({'Sessions': [key], 'Customers': [', '.join(value)]})

    df_final = pd.concat([df_final, new_row], ignore_index=True)

df_final

,Sessions,Customers
0,BRKOPT-2837 - Scaling Enterprise Networks with...,Thomas Delaby (France-IX)
1,BRKDCN-2938 - Intersight Managed Mode (IMM) fo...,Bastien Fine (Adista)
2,BYODCRT-2464 - unknown,zakariaa FODIL (LINKT)
3,BRKNWT-2210 - Predicting Networks are THERE !!,"Mathieu Bonnin (Axione), Frederic Faraux (Clar..."
4,BRKMER-2399 - Meraki Wireless from a Troublesh...,Stanislas de Goriainoff (SEWAN)
...,...,...
155,BRKSP-2637 - Network Automation with Routed Op...,"Jean-Michel Dilly (Adista), Kevin Bourgeaux (A..."
156,IBOCX-1101 - unknown,Pestka Fabien (ALTITUDE INFRA)
157,LTRATO-2001 - unknown,Johan REMY (Adista)
158,BRKSPM-2386 - Getting Ready for Private 5G Dep...,"Rodrigue ESTEVE (Axione), barnabe Louvet (LINK..."


In [136]:
with pd.ExcelWriter('customers_sessions_output.xlsx') as writer:
    df_final.to_excel(writer, index=False)